In [1]:
import pandas as pd

from DataPlug import DataPlug as dp
from SqueezeNet import SqueezeNet as sq

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jackk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
squeeze = sq("SqueezeNet")
dataPlug = dp("DataPlug")


In [3]:
df = pd.read_csv('../data/r_wallstreetbets_posts.csv')
# convert utc time to datetime
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')
df['timestamp'] = df['created_utc'].astype('datetime64[ns]').dt.floor('D')

df.drop('author', inplace=True, axis=1)
df.drop('created_utc', inplace=True, axis=1)
df.drop('id', inplace=True, axis=1)
df.drop('author_flair_text', inplace=True, axis=1)
df.drop('removed_by', inplace=True, axis=1)
df.drop('awarders', inplace=True, axis=1)
df.drop('full_link', inplace=True, axis=1)
df.drop('over_18', inplace=True, axis=1)
df.drop('total_awards_received', inplace=True, axis=1)

df['title'] = df['title'].apply(lambda x: str(x))

display(df)


C:\Users\jackk\AppData\Local\Temp\ipykernel_24336\1872446100.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/r_wallstreetbets_posts.csv')


,title,score,num_comments,timestamp
0,Whats going on with PLTR?,1,2,2021-02-16
1,"Need explanations on Level 2 data for GME, why...",1,2,2021-02-16
2,XRT is being used as a laundry short machine,1,2,2021-02-16
3,Airlines?,1,2,2021-02-16
4,Buy TRXC 🚀,1,2,2021-02-16
...,...,...,...,...
1118858,"EBAY posts higher 1Q net income and revenue, s...",7,4,2012-04-19
1118859,Anyone betting on VVUS and their potential app...,1,0,2012-04-17
1118860,My poorly timed opening position for AAPL earn...,12,21,2012-04-16
1118861,"GOOG - beat estimates, price barely rises.",2,0,2012-04-12


In [4]:
agg_func = {'title': list, 'score': 'mean', 'num_comments': 'mean'}

# Replace NaN values with 'NaN'
#df['total_awards_received'] = df['total_awards_received'].fillna(0)
# Group by date and aggregate
df_new = df.groupby(df['timestamp']).aggregate(agg_func)
display(df_new)
# Join lists of titles, ids, urls, and bodies into one string per date to perform sentiment analysis.

df_new['title'] = df_new['title'].apply(lambda x: '||'.join(x))


display(df_new)

,title,score,num_comments
timestamp,,,
2012-04-11,[Earnings season is here. Place your bets.],13.000000,22.000000
2012-04-12,"[GOOG - beat estimates, price barely rises.]",2.000000,0.000000
2012-04-16,[My poorly timed opening position for AAPL ear...,12.000000,21.000000
2012-04-17,[Anyone betting on VVUS and their potential ap...,1.000000,0.000000
2012-04-19,"[After HGSI spikes 97%, will share price drop ...",5.500000,2.000000
...,...,...,...
2021-02-12,"[When you Can't Afford to Lose, Sell Options! ...",1.002569,21.684764
2021-02-13,"[SatoshiSwap (SAS) is the future!!!!, Strategi...",1.000765,10.059153
2021-02-14,"[Created a cliff for myself to jump off of, I ...",1.000337,10.891326


,title,score,num_comments
timestamp,,,
2012-04-11,Earnings season is here. Place your bets.,13.000000,22.000000
2012-04-12,"GOOG - beat estimates, price barely rises.",2.000000,0.000000
2012-04-16,My poorly timed opening position for AAPL earn...,12.000000,21.000000
2012-04-17,Anyone betting on VVUS and their potential app...,1.000000,0.000000
2012-04-19,"After HGSI spikes 97%, will share price drop a...",5.500000,2.000000
...,...,...,...
2021-02-12,"When you Can't Afford to Lose, Sell Options! +...",1.002569,21.684764
2021-02-13,SatoshiSwap (SAS) is the future!!!!||Strategie...,1.000765,10.059153
2021-02-14,Created a cliff for myself to jump off of||I w...,1.000337,10.891326


In [5]:
dataPlug.df = df_new

print(dataPlug.df.nunique())

display(dataPlug.df)

title           3020
score           2034
num_comments    2279
dtype: int64


,title,score,num_comments
timestamp,,,
2012-04-11,Earnings season is here. Place your bets.,13.000000,22.000000
2012-04-12,"GOOG - beat estimates, price barely rises.",2.000000,0.000000
2012-04-16,My poorly timed opening position for AAPL earn...,12.000000,21.000000
2012-04-17,Anyone betting on VVUS and their potential app...,1.000000,0.000000
2012-04-19,"After HGSI spikes 97%, will share price drop a...",5.500000,2.000000
...,...,...,...
2021-02-12,"When you Can't Afford to Lose, Sell Options! +...",1.002569,21.684764
2021-02-13,SatoshiSwap (SAS) is the future!!!!||Strategie...,1.000765,10.059153
2021-02-14,Created a cliff for myself to jump off of||I w...,1.000337,10.891326


In [8]:
dataPlug.get_price_dataframe()

,timestamp,Open,High,Low,Close,Adj Close,Volume
0,2012-04-11,5.330000,5.380000,5.2350,5.3175,3.603972,19562000
1,2012-04-12,5.325000,5.427500,5.3125,5.3900,3.653110,8414800
2,2012-04-13,5.375000,5.375000,5.2800,5.3075,3.597196,8956000
3,2012-04-16,5.325000,5.452500,5.2425,5.4250,3.676833,13710000
4,2012-04-17,5.455000,5.550000,5.4500,5.5375,3.753081,16022400
...,...,...,...,...,...,...,...
2221,2021-02-08,18.102501,18.165001,14.5050,15.0000,15.000000,102749200
2222,2021-02-09,14.152500,14.250000,11.6300,12.5775,12.577500,107372400
2223,2021-02-10,12.692500,15.707500,11.6375,12.8000,12.800000,145820000
2224,2021-02-11,12.502500,13.830000,12.0550,12.7750,12.775000,52226800


In [9]:
# Merge the dataframes
dataPlug.merge_dataframes()


display(dataPlug.mergedDF)

,timestamp,Open,High,Low,Close,Adj Close,Volume,title,score,num_comments
0,2012-04-11,5.330000,5.380000,5.2350,5.3175,3.603972,19562000,Earnings season is here. Place your bets.,13.000000,22.000000
1,2012-04-12,5.325000,5.427500,5.3125,5.3900,3.653110,8414800,"GOOG - beat estimates, price barely rises.",2.000000,0.000000
2,2012-04-16,5.325000,5.452500,5.2425,5.4250,3.676833,13710000,My poorly timed opening position for AAPL earn...,12.000000,21.000000
3,2012-04-17,5.455000,5.550000,5.4500,5.5375,3.753081,16022400,Anyone betting on VVUS and their potential app...,1.000000,0.000000
4,2012-04-19,5.562500,5.635000,5.5325,5.5725,3.776802,14128400,"After HGSI spikes 97%, will share price drop a...",5.500000,2.000000
...,...,...,...,...,...,...,...,...,...,...
2140,2021-02-08,18.102501,18.165001,14.5050,15.0000,15.000000,102749200,Me on the RH||Can you Help me giving the name ...,1.085725,10.603988
2141,2021-02-09,14.152500,14.250000,11.6300,12.5775,12.577500,107372400,Cindicator Capital Wants to Hire r/WallStreetB...,1.000829,13.947142
2142,2021-02-10,12.692500,15.707500,11.6375,12.8000,12.800000,145820000,Why the fuck isn’t anyone talking about then G...,6.014227,20.014591
2143,2021-02-11,12.502500,13.830000,12.0550,12.7750,12.775000,52226800,Who loves a Pi ?||Does anyone know about Atlis...,1.207335,17.038586


In [11]:
positive = []
negative = []
neutral = []
compound = []

import time

begin = time.time() # timer for entire process

for i in range(0, len(dataPlug.mergedDF['title'])):
    start = time.time() # timer for each iteration

    if i == 2131:
        positive.append(0)
        negative.append(0)
        neutral.append(0)
        compound.append(0)
        continue

    text = dataPlug.mergedDF['title'][i]
    clean_text = squeeze.clean_text(text)
    score = squeeze.sentiment_analysis(clean_text)
    # Add scores to array
    positive.append(score['pos'])
    negative.append(score['neg'])
    neutral.append(score['neu'])
    compound.append(score['compound'])



    end = time.time() # end timer for each iteration

    print(f"Completed {i} of {len(dataPlug.mergedDF['title'])} posts | {end - start:0.4f} seconds | {end - begin:0.4f} seconds total")

print(f"Sentiment analysis took {end - begin:0.4f} seconds total")



Completed 0 of 2145 posts | 0.0130 seconds | 0.0130 seconds total
Completed 1 of 2145 posts | 0.0120 seconds | 0.0250 seconds total
Completed 2 of 2145 posts | 0.0130 seconds | 0.0380 seconds total
Completed 3 of 2145 posts | 0.0120 seconds | 0.0500 seconds total
Completed 4 of 2145 posts | 0.0110 seconds | 0.0610 seconds total
Completed 5 of 2145 posts | 0.0110 seconds | 0.0720 seconds total
Completed 6 of 2145 posts | 0.0100 seconds | 0.0820 seconds total
Completed 7 of 2145 posts | 0.0110 seconds | 0.0930 seconds total
Completed 8 of 2145 posts | 0.0110 seconds | 0.1040 seconds total
Completed 9 of 2145 posts | 0.0120 seconds | 0.1160 seconds total
Completed 10 of 2145 posts | 0.0130 seconds | 0.1290 seconds total
Completed 11 of 2145 posts | 0.0121 seconds | 0.1411 seconds total
Completed 12 of 2145 posts | 0.0110 seconds | 0.1521 seconds total
Completed 13 of 2145 posts | 0.0110 seconds | 0.1631 seconds total
Completed 14 of 2145 posts | 0.0110 seconds | 0.1741 seconds total
Compl

KeyboardInterrupt: 

In [ ]:
# Using DataFrame.insert() to add the sentiment columns to the dataframe
dataPlug.mergedDF.insert(8, "Compound_Sentiment", compound, True)
dataPlug.mergedDF.insert(8, "Negative_Sentiment", negative, True)
dataPlug.mergedDF.insert(8, "Neutral_Sentiment", neutral, True)
dataPlug.mergedDF.insert(8, "Positive_Sentiment", positive, True)
print('Done!')

In [ ]:
# Drop the column with the weird data issue
dataPlug.mergedDF = dataPlug.mergedDF.drop(labels=2131, axis=0)

In [ ]:
display(dataPlug.mergedDF)
dataPlug.mergedDF.to_csv('../data/mergedDF.csv')
print(f'saved to ../data/mergedDF.csv')

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score


In [ ]:
prices = dataPlug.mergedDF['High'].tolist()
y = []
for i in prices:
        y.append(float(i))
volume_y = dataPlug.mergedDF['Volume'].tolist()


dataPlug.mergedDF = dataPlug.mergedDF.drop('Open', axis=1)
dataPlug.mergedDF = dataPlug.mergedDF.drop('High', axis=1)
dataPlug.mergedDF = dataPlug.mergedDF.drop('Low', axis=1)
dataPlug.mergedDF = dataPlug.mergedDF.drop('Close', axis=1)
dataPlug.mergedDF = dataPlug.mergedDF.drop('Adj Close', axis=1)
dataPlug.mergedDF = dataPlug.mergedDF.drop('Volume', axis=1)


In [ ]:
dataPlug.mergedDF = dataPlug.mergedDF.drop('timestamp', axis=1)
display(dataPlug.mergedDF)

In [ ]:
dataPlug.mergedDF = dataPlug.mergedDF.drop('title', axis=1)
dataPlug.mergedDF = dataPlug.mergedDF.drop('body', axis=1)
display(dataPlug.mergedDF)

In [ ]:
X = dataPlug.mergedDF
# split data into test and training sets
X_train, X_test, y_train_list, y_test_list = train_test_split(X, y, test_size=0.5, random_state=3)
y_train = []
for item in y_train_list:
    y_train.append(int(item))

y_test = []
for item in y_test_list:
    y_test.append(int(item))


In [ ]:
# train Bernoulli Naive Bayes model on the training set
clf = BernoulliNB()
clf.fit(X_train.astype('float'), y_train)


In [ ]:
import sklearn.metrics as metrics

# predict labels for training and test sets
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

train_score = metrics.accuracy_score(y_train, y_train_pred)




print("Training score:", train_score)



In [ ]:
from sklearn.metrics import classification_report
target_names = ['High']
print(classification_report(y_test, y_test_pred.astype('int'), target_names=target_names))
